# Projeto 1 - Ciência dos Dados

Nome: **Gianluca Lazzaris Giudici**

Nome: **Pietro Abe Seixas**

# Proposta

Neste projeto a partir de um analisador de Naive-Bayes, a proposta é criar um identificador de sentimento.

___
Carregando algumas bibliotecas:

In [90]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path
import json
import re
import string
import functools
import operator
from random import shuffle

try:
    import emoji
except:
    !pip install emoji --upgrade
    import emoji

In [91]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/macbookpro/Documents/Git/cdados/Projeto_1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [93]:
dados_treinamento = pd.read_excel('mcdonalds.xlsx')
dados_teste = pd.read_excel("mcdonalds.xlsx", "Teste")
dados_treinamento.head(5)

,Treinamento,Classificação
0,aqui no mcdonalds pra tentar matar a minha rai...,0
1,@rafaamrtz @mcdonalds_br quer ir agoraaa??? 🤭,0
2,@ialanau @zipzop26169213 @mcdonalds_br aonde e...,0
3,@mcdonalds_br postou errado né estagiario,0
4,@igorneumann27 @mcdonalds_br um big tasty com ...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

**O produto escolhido foi a empresa "McDonalds", onde separamos em dois grupos: relevantes, sendo esse identificado pelo numero 1 na coluna de classificação do excel, onde selecionamos tweets que possam ser utéis para a empresa escolhida melhorar seu atendimento em alguma de suas aréas; irrelevantes, sendo esse identificado pelo numero 0 na coluna de classificação do excel, onde foi selecionado tweets que não ajudam em nada o McDonalds a melhor em algo no seu atendimento.** 

### Limpeza dos dados.

Podemos perceber, que os tweets que foram obtidos possuem pontuação, emojis, e alguns caractéres como **\n**, que irão poluir a análise.

Com isso, resolvemos utilizar algumas funções para fazer a limpeza dos dados, facilitando a análise.

Primeiro, separando os dados para facilitar a visualização:

In [95]:
dados_treinamento_relevantes = dados_treinamento.loc[dados_treinamento.Classificação == 1]
dados_treinamento_nao_relevantes = dados_treinamento.loc[dados_treinamento.Classificação == 0]

treinamento_string_relevantes = ""
treinamento_string_nao_relevantes = ""

Construindo funções:

In [56]:
#devido a grande quantidade de risadas "kkk" com diferentes tamanhos, iremos padronizar as risadas.
def verifica_k(palavra):
    for letra in palavra:
        if letra != "k":
            return False
    return True

def limpa_texto(texto):
    
    
    texto_separa_emoji = emoji.get_emoji_regexp().split(texto)
    texto_separa_espaco = [substr.split() for substr in texto_separa_emoji]
    texto_separado = functools.reduce(operator.concat, texto_separa_espaco)
    texto_separado = " ".join(palavra for palavra in texto_separado)
    
    texto = " ".join(palavra for palavra in texto_separado.split() if not palavra.startswith("https"))
    
    pontuacao = "[;;?.!-]"
    padronizado = re.compile(pontuacao)
    
    texto_padronizado = re.sub(padronizado, " ", texto)
    texto_padronizado = texto_padronizado.lower()
    texto_saida = " "
    
    for palavra in texto_padronizado.split():
        if verifica_k(palavra):
            texto_saida = texto_saida + "hahaha" #substituindo "kkk" por "hahaha"
        else:
            texto_saida = texto_saida + " " + palavra
    
    return texto_saida

Usando as funções acima nos data frames:

In [42]:
for frase in dados_treinamento_relevantes.Treinamento:
    treinamento_string_relevantes = treinamento_string_relevantes + frase
for frase in dados_treinamento_nao_relevantes.Treinamento:
    treinamento_string_nao_relevantes = treinamento_string_nao_relevantes + frase
    
limpa_texto(treinamento_string_relevantes)
limpa_texto(treinamento_string_nao_relevantes)

treinamento_pandas_relevantes = pd.Series(treinamento_string_relevantes.split()).value_counts(True)
treinamento_pandas_nao_relevantes = pd.Series(treinamento_string_nao_relevantes.split()).value_counts(True)

treinamento_pandas_relevantes_total = pd.Series(treinamento_string_relevantes.split()).value_counts()
treinamento_pandas_nao_relevantes_total = pd.Series(treinamento_string_nao_relevantes.split()).value_counts()

tamanho_tweets_relevantes = len(treinamento_pandas_relevantes_total)
tamanho_tweets_nao_relevantes = len(treinamento_pandas_nao_relevantes_total)

tamanho_tweets_total = tamanho_tweets_nao_relevantes + tamanho_tweets_relevantes

treinamento_pandas_join = pd.Series((treinamento_string_relevantes + treinamento_string_nao_relevantes).split()).value_counts(True)

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [43]:
prob_nao_relevantes = tamanho_tweets_nao_relevantes / tamanho_tweets_total
prob_relevantes = tamanho_tweets_relevantes / tamanho_tweets_total

In [96]:
## Montando o Classificador ##

def classificador_tweets (tweets = "", alpha = 1, V = 1250):
    
    ## limpando os tweets
    
    texto = limpa_texto(tweets)
    
    ## avaliando a probabilidade, onde foi feito o uso de log para facilitar a vizualização dos dados.
    
    contador_relevantes = 0
    contador_nao_relevantes = 0
    t_relevantes = tamanho_tweets_relevantes + alpha * V
    t_nao_relevantes = tamanho_tweets_nao_relevantes + alpha * V
    
    for palavra in texto.split():
        
        if palavra in treinamento_pandas_relevantes:
            contador_relevantes += np.log((treinamento_pandas_relevantes_total[palavra] + alpha) / t_relevantes)
        else:
            contador_relevantes += np.log(alpha / t_relevantes)
        if palavra in treinamento_pandas_nao_relevantes:
            contador_nao_relevantes += np.log((treinamento_pandas_nao_relevantes_total[palavra] + alpha) / t_nao_relevantes)
        else:
            contador_nao_relevantes += np.log(alpha / t_nao_relevantes)
            
    return contador_relevantes > contador_nao_relevantes
    

___
### Verificando a performance do Classificador

Agora vamos usar a função classificador_tweets para verificar a performance.

In [97]:
classificacao = [int(classificador_tweets(str(tweets), 1, 1250)) for tweets in dados_teste.Teste]

dados_teste["bayes"] = classificacao

acuracia = {"Verdadeiro-Verdadeiro":0,
           "Verdadeiro-Falso":0,
           "Falso-Verdadeiro":0,
           "Falso-Falso":0}

for tweets, classificacao_2 in zip(dados_teste.Classificação, dados_teste.bayes):
    if tweets and classificacao_2:
        acuracia['Verdadeiro-Verdadeiro'] += 1
    elif tweets and not classificacao_2:
        acuracia['Verdadeiro-Falso'] += 1
    elif not tweets and classificacao_2:
        acuracia['Falso-Verdadeiro'] += 1
    elif not tweets and not classificacao_2:
        acuracia['Falso-Falso'] += 1
        
acuracia_normalizada = {}
soma = sum(acuracia.values())
for k,v in acuracia.items():
    acuracia_normalizada[k] = v/soma
acuracia_normalizada['Acuracia-Final'] = str(round((acuracia_normalizada['Verdadeiro-Verdadeiro']+acuracia_normalizada['Falso-Falso'])*100, 3)) + '%'
lista.append(acuracia_normalizada['Acuracia-Final'])
A = dados_teste.Classificação.value_counts(True)
print("Podemmos perceber, que, da nossa validação {1}% são relevantes e {0}% não, de acordo com nossa avaliação manual.".format(round(A[0]*100,2),round(A[1]*100, 2)))
acuracia_normalizada

Podemmos perceber, que, da nossa validação 40.0% são relevantes e 60.0% não, de acordo com nossa avaliação manual.


{'Verdadeiro-Verdadeiro': 0.12777777777777777,
 'Verdadeiro-Falso': 0.2722222222222222,
 'Falso-Verdadeiro': 0.11666666666666667,
 'Falso-Falso': 0.48333333333333334,
 'Acuracia-Final': '61.111%'}

## Usando o SKLearn:

In [98]:
S = dados_treinamento
S_teste = dados_teste

Xt_train = S['Treinamento']
y_train = S['Classificação'] == 1

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

count = CountVectorizer()
X_train = count.fit_transform(Xt_train)

model = MultinomialNB(alpha=1)
model.fit(X_train, y_train)

Xt_test = [str(h) for h in S_teste["Teste"]]

y_test = S_teste['Classificação'] == 1

X_test = count.transform(Xt_test)

y_pred = model.predict(X_test)


"A acurácia ideal seria: {0}%".format(round(accuracy_score(y_test, y_pred)*100,3))


'A acurácia ideal seria: 64.444%'

___
### Concluindo

Após a analise de dados, podemos concluir que nosso classificador Naive-Bayes é horrível, pois se presumirmos que tudo é irrelevante ele só ganha por 1,1%. Isso pode ser explicado por; A baixa quantidade de tweets de treinamento, uma grande dificuldade em classificar se um tweet é positivo ou negativo, algo muito ambiguo no assunto, ou também, porque um mesmo verbo pode ser tanto negativo quanto positivo dificultando ainda mais nosso classificador.

### Por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets?

Se treinassemos o modelo com os dados que o próprio modelo produz não teria sentido, pois fazendo isso ele bsicamente só decoraria os resultados obtidos anteriormente. Treinar nosso modelo com outro modelo até seria possível, mas seria algo muito mais complexo que envolveria outras técnicas que não aprendemos, como apreendizado cooperativo.

### Outros usos para o classificador de Naive_Bayes.

O classificador de Naive_Bayes tem diversos usos. Alguns exemplos são; classificação de spam de emails, diagnostico de alguma doença, previsão do tempo e diversos outros usos.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

[Naive_Bayes Classification](https://www.datageeks.com.br/naive-bayes/) **Outra fonte utilizada**